In [45]:
import os
import numpy as np
import pandas as pd
from datetime import datetime
from dotenv import load_dotenv
from multielo import MultiElo, Player, Tracker
from mktools.get_data import load_data_pd
from alive_progress import alive_it
from mktools.validate_data import validate_bad_uids
import plotly.express as px

load_dotenv()

True

In [46]:
df = load_data_pd(
    sheet_name="data_main",
    sheet_id=os.environ["SHEET_ID"],
    usecols=[
        "UID",
        "SUID",
        "NAME",
        "CHARACTER",
        "MAP",
        "PLACE",
        "PLAYERS",
        "DATE",
        "SEASON",
    ],
)

df["DATE"] = pd.to_datetime(df["DATE"])

invalid, valid = validate_bad_uids(df=df, return_valid=True)

In [47]:
valid

,UID,SUID,NAME,CHARACTER,MAP,PLACE,PLAYERS,DATE,SEASON
0,1,1,Cole,Toad,Sherbet Land,4,4,2021-09-20 00:00:00,0
1,1,1,Connor,Yoshi,Sherbet Land,2,4,2021-09-20 00:00:00,0
2,1,1,Cooper,Peach,Sherbet Land,1,4,2021-09-20 00:00:00,0
3,1,1,Triston,Bowser,Sherbet Land,3,4,2021-09-20 00:00:00,0
4,2,1,Cole,Toad,Kalimari Desert,4,4,2021-09-20 00:00:00,0
...,...,...,...,...,...,...,...,...,...
20914,6368,742,Domingo,Toad,Banshee Boardwalk,4,4,2024-07-22 20:38:49,11
20915,6369,742,Cooper,D.K.,Koopa Troopa Beach,1,4,2024-07-22 21:20:25,11
20916,6369,742,Cole,Yoshi,Koopa Troopa Beach,2,4,2024-07-22 21:20:25,11
20917,6369,742,Blake,Peach,Koopa Troopa Beach,3,4,2024-07-22 21:20:25,11


In [48]:
valid[valid["NAME"].isna()]

,UID,SUID,NAME,CHARACTER,MAP,PLACE,PLAYERS,DATE,SEASON


In [49]:
vdf = valid.copy()


msk = vdf.duplicated(subset=["UID", "NAME"], keep=False)

dupe_names = vdf[msk]

dupe_names

,UID,SUID,NAME,CHARACTER,MAP,PLACE,PLAYERS,DATE,SEASON
484,133,12,Connor,Peach,Toad's Turnpike,1,4,2021-10-01,0
485,133,12,Connor,Toad,Toad's Turnpike,4,4,2021-10-01,0
1237,348,27,Cole,Yoshi,Bowser's Castle,1,2,2021-11-13,0
1238,348,27,Connor,Peach,Bowser's Castle,2,2,2021-11-13,0
1239,348,27,Cole,Peach,Koopa Troopa Beach,2,2,2021-11-13,0
...,...,...,...,...,...,...,...,...,...
18869,5791,694,Cooper,Peach,D.K.'s Jungle,2,3,2024-06-04,10
20270,6179,725,Konnor,Peach,Mario Raceway,1,2,2024-07-05,11
20271,6179,725,Cooper,Yoshi,Mario Raceway,2,2,2024-07-05,11
20272,6179,725,Konnor,Yoshi,Banshee Boardwalk,1,2,2024-07-05,11


In [50]:
invalid_names = vdf[vdf["UID"].isin(dupe_names["UID"])]

In [51]:
valid_valid = vdf[~vdf["UID"].isin(dupe_names["UID"])].copy().reset_index(drop=True)

In [52]:
valid_valid[valid_valid["NAME"].isna()]

,UID,SUID,NAME,CHARACTER,MAP,PLACE,PLAYERS,DATE,SEASON


In [53]:
place_msk = valid_valid.duplicated(subset=["UID", "PLACE"], keep=False)

dupe_places = valid_valid[place_msk]

In [54]:
invalid_places = valid_valid[valid_valid["UID"].isin(dupe_places["UID"])]

invalid_places

,UID,SUID,NAME,CHARACTER,MAP,PLACE,PLAYERS,DATE,SEASON
120,33,3,Blake,Bowser,Wario Stadium,2,4,2021-09-22,0
121,33,3,Connor,Toad,Wario Stadium,1,4,2021-09-22,0
122,33,3,Cooper,D.K.,Wario Stadium,4,4,2021-09-22,0
123,33,3,Regan,Peach,Wario Stadium,2,4,2021-09-22,0
1068,300,24,Blake,Bowser,Wario Stadium,1,4,2021-10-10,0
1069,300,24,Cooper,Toad,Wario Stadium,2,4,2021-10-10,0
1070,300,24,Matt,Peach,Wario Stadium,2,4,2021-10-10,0
1071,300,24,Regan,Yoshi,Wario Stadium,3,4,2021-10-10,0
2626,776,63,Regan,Toad,Wario Stadium,1,4,2021-12-15,1
2627,776,63,Cooper,Peach,Wario Stadium,2,4,2021-12-15,1


In [55]:
valid_valid_valid = (
    valid_valid[~valid_valid["UID"].isin(invalid_places["UID"])]
    .copy()
    .reset_index(drop=True)
)

In [56]:
valid_valid_valid[valid_valid_valid["NAME"].isna()]

,UID,SUID,NAME,CHARACTER,MAP,PLACE,PLAYERS,DATE,SEASON


In [57]:
v_invalid, v_valid = validate_bad_uids(df=valid_valid_valid, return_valid=True)

In [58]:
v_invalid

,UID,SUID,NAME,CHARACTER,MAP,PLACE,PLAYERS,DATE,SEASON


In [59]:
v_valid[v_valid["NAME"].isna()]

,UID,SUID,NAME,CHARACTER,MAP,PLACE,PLAYERS,DATE,SEASON


In [60]:
oob_two_p = v_valid[(~v_valid["PLACE"].isin([1, 2])) & (v_valid["PLAYERS"] == 2)].copy()

oob_three_p = v_valid[
    (~v_valid["PLACE"].isin([1, 2, 3])) & (v_valid["PLAYERS"] == 3)
].copy()

oob_four_p = v_valid[
    (~v_valid["PLACE"].isin([1, 2, 3, 4])) & (v_valid["PLAYERS"] == 4)
].copy()

oob_concat = pd.concat([oob_two_p, oob_three_p, oob_four_p])

oob_invalid = v_valid[v_valid["UID"].isin(oob_concat["UID"])]

oob_invalid

,UID,SUID,NAME,CHARACTER,MAP,PLACE,PLAYERS,DATE,SEASON


## ALL ELO

In [61]:
tdf = (
    v_valid[(v_valid["PLAYERS"] == 2)]
    .drop(columns=["SUID", "CHARACTER", "PLAYERS", "SEASON", "MAP"])
    .sort_values(by=["UID", "DATE", "PLACE"])
    .copy()
    .reset_index(drop=True)
)


pdf = (
    tdf.pivot(index="UID", columns="PLACE", values="NAME")
    .reset_index()
    .rename(columns={1: "1st", 2: "2nd", 3: "3rd", 4: "4th"})
    .merge(tdf[["UID", "DATE"]], on="UID", how="inner", validate="1:m")
    .groupby("UID")
    .first()
    # .set_index("DATE")
    # .reset_index()
    .rename(columns={"DATE": "date"})
)

In [62]:
pdf

,1st,2nd,date
UID,,,
36,Cooper,Triston,2021-09-22 00:00:00
75,Regan,Cooper,2021-09-27 00:00:00
104,Blake,Connor,2021-09-29 00:00:00
105,Blake,Connor,2021-09-29 00:00:00
111,Cooper,Triston,2021-09-30 00:00:00
...,...,...,...
6276,Chandler,Antonio,2024-07-14 00:00:00
6293,Cooper,Konnor,2024-07-16 00:00:00
6307,Cooper,Regan,2024-07-18 21:05:00


In [63]:
pdf

,1st,2nd,date
UID,,,
36,Cooper,Triston,2021-09-22 00:00:00
75,Regan,Cooper,2021-09-27 00:00:00
104,Blake,Connor,2021-09-29 00:00:00
105,Blake,Connor,2021-09-29 00:00:00
111,Cooper,Triston,2021-09-30 00:00:00
...,...,...,...
6276,Chandler,Antonio,2024-07-14 00:00:00
6293,Cooper,Konnor,2024-07-16 00:00:00
6307,Cooper,Regan,2024-07-18 21:05:00


In [64]:
pi = pdf.reset_index().set_index("date")

pi["diff"] = pi.index.diff()

pi = pi.reset_index().fillna(pd.Timedelta(0))

pi

,date,UID,1st,2nd,diff
0,2021-09-22 00:00:00,36,Cooper,Triston,0 days 00:00:00
1,2021-09-27 00:00:00,75,Regan,Cooper,5 days 00:00:00
2,2021-09-29 00:00:00,104,Blake,Connor,2 days 00:00:00
3,2021-09-29 00:00:00,105,Blake,Connor,0 days 00:00:00
4,2021-09-30 00:00:00,111,Cooper,Triston,1 days 00:00:00
...,...,...,...,...,...
1073,2024-07-14 00:00:00,6276,Chandler,Antonio,0 days 00:00:00
1074,2024-07-16 00:00:00,6293,Cooper,Konnor,2 days 00:00:00
1075,2024-07-18 21:05:00,6307,Cooper,Regan,2 days 21:05:00
1076,2024-07-21 16:32:47,6347,Matt,Cooper,2 days 19:27:47


In [65]:
pi[pi["diff"] < pd.Timedelta(1)]

,date,UID,1st,2nd,diff
0,2021-09-22,36,Cooper,Triston,0 days
3,2021-09-29,105,Blake,Connor,0 days
7,2021-10-02,137,Blake,Connor,0 days
8,2021-10-02,138,Connor,Blake,0 days
9,2021-10-02,139,Regan,Cooper,0 days
...,...,...,...,...,...
1068,2024-07-13,6262,Cooper,Matt,0 days
1069,2024-07-13,6264,Chandler,Cooper,0 days
1070,2024-07-13,6266,Cooper,Matt,0 days
1071,2024-07-13,6269,Cooper,Triston,0 days


In [66]:
bad_dates = pi[pi["diff"].dt.total_seconds() < 0]

bad_dates

,date,UID,1st,2nd,diff


In [ ]:
tdf[tdf["DATE"].isin(bad_dates["date"])]

In [ ]:
pi["diff"].dt.total_seconds() < 0 

In [ ]:
pdf[pdf["date"].isna()]

In [ ]:
p = tdf.pivot(index="UID", columns="PLACE", values="NAME")

p[p.isna().any(axis=1)]

In [ ]:
pdf.isna().any(axis=1)

In [ ]:
pdf[pdf.isna().any(axis=1)]

In [ ]:
exp_elo = MultiElo(score_function_base=2)
tracker = Tracker(elo_rater=exp_elo)
tracker.process_data(pdf)

rdf = tracker.get_current_ratings()

In [ ]:
rdf.sort_values(["rating", "n_games"], ascending=[False, False])

In [ ]:
filter_df = rdf[rdf["n_games"] >= 10]

In [ ]:
track_df = tracker.get_history_df()

track_df = track_df[track_df["player_id"].isin(filter_df["player_id"])].reset_index(
    drop=True
)


all_fig = px.line(data_frame=track_df, x="date", y="rating", color="player_id", height=800, markers=True)

all_fig

## SEASON BY TYPE

In [ ]:
seasons_list = [10, 11]

In [ ]:
rated_seasons = []
tracks_list = []


for season in alive_it(
    it=seasons_list,
    total=len(seasons_list),
    theme="classic",
):

    tdf = (
        v_valid[(v_valid["PLAYERS"] == 4) & (v_valid["SEASON"] == season)]
        .drop(columns=["SUID", "CHARACTER", "PLAYERS", "SEASON", "MAP"])
        .sort_values(by=["UID", "PLACE"])
        .copy()
        .reset_index(drop=True)
    )

    pdf = (
        tdf.pivot(index="UID", columns="PLACE", values="NAME")
        .reset_index()
        .rename(columns={1: "1st", 2: "2nd", 3: "3rd", 4: "4th"})
        .merge(tdf[["UID", "DATE"]], on="UID", how="inner", validate="1:m")
        .groupby("UID")
        .first()
        .set_index("DATE")
        .reset_index()
        .rename(columns={"DATE": "date"})
    )

    exp_elo = MultiElo(score_function_base=1.5)
    tracker = Tracker(elo_rater=exp_elo)
    tracker.process_data(pdf)

    rdf = tracker.get_current_ratings()

    tr_df = tracker.get_history_df()

    rdf["SEASON"] = season

    rated_seasons.append(rdf)

    tracks_list.append(tr_df)

In [ ]:
tracks_list[0]

In [ ]:
all_fig = px.line(data_frame=tracks_list[0], x="date", y="rating", color="player_id", height=800, markers=True)

all_fig

In [ ]:
# rated_seasons = []

# for season in alive_it(
#     it=v_valid["SEASON"].unique(),
#     total=v_valid["SEASON"].unique().shape[0],
#     theme="classic",
# ):

#     tdf = (
#         v_valid[(v_valid["PLAYERS"] == 4) & (v_valid["SEASON"] == season)]
#         .drop(columns=["SUID", "CHARACTER", "PLAYERS", "SEASON", "MAP"])
#         .sort_values(by=["UID", "PLACE"])
#         .copy()
#         .reset_index(drop=True)
#     )

#     pdf = (
#         tdf.pivot(index="UID", columns="PLACE", values="NAME")
#         .reset_index()
#         .rename(columns={1: "1st", 2: "2nd", 3: "3rd", 4: "4th"})
#         .merge(tdf[["UID", "DATE"]], on="UID", how="inner", validate="1:m")
#         .groupby("UID")
#         .first()
#         .set_index("DATE")
#         .reset_index()
#         .rename(columns={"DATE": "date"})
#     )

#     exp_elo = MultiElo(score_function_base=1.5)
#     tracker = Tracker(elo_rater=exp_elo)
#     tracker.process_data(pdf)

#     rdf = tracker.get_current_ratings()

#     rdf["SEASON"] = season

#     rated_seasons.append(rdf)

In [ ]:
rated_seasons[-1]

In [ ]:
rated_seasons[0]

In [ ]:
pd.concat(rated_seasons).sort_values(by=["SEASON", "rank"]).reset_index(drop=True)

In [ ]:
track_df = tracker.get_history_df()

In [ ]:
tdf

In [ ]:
df

## 4 Player ELO

In [ ]:
df = load_data_pd(
    sheet_name="data_main",
    sheet_id=os.environ["SHEET_ID"],
    usecols=[
        "UID",
        "SUID",
        "NAME",
        "CHARACTER",
        "MAP",
        "PLACE",
        "PLAYERS",
        "DATE",
        "SEASON",
    ],
)

df["DATE"] = pd.to_datetime(df["DATE"]).astype(str)

invalid, valid = validate_bad_uids(df=df, return_valid=True)

In [ ]:
tdf = (
    valid[(valid["PLAYERS"] == 4) & (valid["SEASON"] == 11)]
    .drop(columns=["SUID", "CHARACTER", "PLAYERS", "SEASON", "MAP"])
    .sort_values(by=["UID", "PLACE"])
    .copy()
    .reset_index(drop=True)
)

pdf = (
    tdf.pivot(index="UID", columns="PLACE", values="NAME")
    .reset_index()
    .rename(columns={1: "1st", 2: "2nd", 3: "3rd", 4: "4th"})
    .merge(tdf[["UID", "DATE"]], on="UID", how="inner", validate="1:m")
    .groupby("UID")
    .first()
    .set_index("DATE")
    .reset_index()
    .rename(columns={"DATE": "date"})
)

exp_elo = MultiElo(score_function_base=2)
tracker = Tracker(elo_rater=exp_elo)
tracker.process_data(pdf)

rdf = tracker.get_current_ratings()

In [ ]:
# filter_df = rdf[rdf["n_games"] > 1].copy()


filter_df = rdf.copy()
filter_df

In [ ]:
import altair as alt

track_df = tracker.get_history_df()

track_df = track_df[track_df["player_id"].isin(filter_df["player_id"])].reset_index(
    drop=True
)

alt.Chart(track_df).mark_line(point=True).encode(
    x="date:T",
    y=alt.Y("rating:Q", scale=alt.Scale(zero=False)),
    color="player_id:N",
).properties(width=1000, height=500, title="Season 11 ELO")

In [ ]:
track_df.head()

In [ ]:
fig = px.line(data_frame=track_df, x="date", y="rating", color="player_id", height=800, markers=True)

fig

In [ ]:
fig.write_html(r"C:\Users\Cooper\sandbox\mkstream\assets\fig\s11_4p_elo.html")

In [ ]:
track_df.to_clipboard()